Long Short Term Model

1. Make sure all packages are installed

*    pip install numpy
*   pip install pandas
*   pip install os-sys
*    pip install matplotlib
*   pip install seaborn
*   pip install scikit-learn
*   pip install keras
*   pip install tensorflow*

2. Collected Data from sensor py to this file 

Potential Issues:
 * Should AccelerationX, Y, Z still be kept in dataframe?
    inputshape is 7, not 5 
 * not sure why we have to reshape features twice 
 * need cat.codes and Label_Encoder, or just one?
 * would y.to_categorical be better? 

In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn 

In [2]:
#https://www.kaggle.com/harnoor343/fall-detection-accelerometer-data
#Link to chen hua min's code: https://www.kaggle.com/code/mmttdebbcc/notebook9689a93a64  
# create a new directory called 'kaggle_normalized', ignore it if it already exists
os.makedirs('./normalized', exist_ok=True)

labels = ['runfall', 'downSit', 'freeFall', 'runSit', 'walkFall', 'walkSit']
for label in labels:
    #read all csv files in the director "./kaggle_data/" + l, delimit by ';'
    #this is different from chen human min project
    files = glob.glob('./archive/' + label + '/*.csv')
    #assign l to to df column 'Label'
    df = pd.concat([pd.read_csv(f, sep=';') for f in files], ignore_index=True)
    # assign l to df column 'Label'
    df['Label'] = label
    # save columns 'DeviceOrientation', 'AccelerationX', 'AccelerationY', 'AccelerationZ', 'Label' to a new csv file
    df[['DeviceOrientation', 'AccelerationX', 'AccelerationY', 'AccelerationZ', 'Label']].to_csv('./normalized/' + label + '.csv', index=False)

df = pd.concat([pd.read_csv(f) for f in glob.glob('./normalized/*.csv')], ignore_index = True)
columns = ['DeviceOrientation', 'AccelerationX', 'AccelerationY', 'AccelerationZ', 'Label']




In [3]:

# for each column in 'Accerlation', 'AccelerationY', 'AccelerationZ', and a new column that is the fft of the original column
for c in ['AccelerationX', 'AccelerationY', 'AccelerationZ']:
    df[c + '_fft'] = np.fft.fft(df[c])

In [ ]:
# plot the first 100 column 'AccelerationX' and 'AccelerationX_fft'
df[['AccelerationX', 'AccelerationY', 'AccelerationZ']].iloc[:100].plot()

# Versus Head (the same)
#df[['AccelerationX_fft', 'AccelerationY_fft', 'AccelerationZ_fft']].head(100).plot()
df[['AccelerationX', 'AccelerationY', 'AccelerationZ']].head(100).plot()

Debugging Code (Mardown Text)

 print the shape of the dataframe

              print(df.shape) 

 print each column's name in the dataframe
 
              print(df.columns)

(202599, 8)
Index(['DeviceOrientation', 'AccelerationX', 'AccelerationY', 'AccelerationZ',
       'Label', 'AccelerationX_fft', 'AccelerationY_fft', 'AccelerationZ_fft'],
      dtype='object')

In [ ]:
# convert categorical data to numerical data
df['Label'] = df['Label'].astype('category')
df['Label'] = df['Label'].cat.codes
# df.head()

# convert column `DeviceOrientation` to numerical data
df['DeviceOrientation'] = df['DeviceOrientation'].astype('category')
df['DeviceOrientation'] = df['DeviceOrientation'].cat.codes
df.head()

x = np.array(df.drop(['Label'], 1)) #drops row 'Label'
y = np.array(df["Label"])

import sklearn.model_selection
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x,y,test_size = 0.3)


In [ ]:
#LTSM version
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder

# encode the target. 
encoder = LabelEncoder()
    #LabelEncoder encodes Encode target labels with value between 0 and n_classes-1.
    # This transformer should be used to encode target values, *i.e.* y, and not the input X
encoder.fit(y_train)
target = encoder.transform(y_train) #tranform to normalised encoding 
target = np.array(target).reshape(-1,1) # (-1,1) because our data has a single feature, and a 'n' amount of rows
print(target)

#build the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(x_train.shape[1], 1)))
model.add(Dense(6, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#reshape the features for the LSTM layer
# don't completely understand this part
features = np.array(x_train).reshape(x_train.shape[0], x_train.shape[1], 1)
#x_train.shape:  141819  x_train.shape[1]:  7  1:  1

#train the model
model.fit(features, target, epochs=1, batch_size=20, verbose=1, validation_split = 0.2)

test_loss, test_Acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_Acc)
model.summary()

0.6795163154602051 
batch_Size = 24 epochs = 15
batch_Size = 20, epochs = 20, two new Relu layers
validation_split = 0.1
epochs = 25, batch_Size = 15